<br />

<div style="text-align: center;">
<font size="7">課題１レポート</font>
</div>
<br />
<div style="text-align: right;">
<font size="4">Masaki Ozawa</font>
</div>

<br />

# 前処理

### 入室時刻に最も近い時刻に検査している血液データとバイタルデータの抽出
##### ①必要な項目を血液データとバイタルデータから抽出する
##### ②それぞれの項目ごとにpIDとdata, Timeと一緒に抽出し、pIDをkeyとして病名ファイルと結合する
##### ③ent_dateと検査時のdateの時間差を計算する
##### ④欠損値を排除したのち、上の計算結果の最小値を取得することで、入室時に最も近い検査値を取得する
##### ⑤それぞれの項目に対して上の操作を行なったのち、全て結合する
### 病名が二つ以上同じ行に入力されている場合、一つ一つ異なる行に分割する
##### Excelにて病名を分ける作業を行なった

In [1]:
import numpy as np
import pandas as pd

In [2]:
#病名データ、血液データ、バイタルデータの３つのcsvファイルの読み込み
patient_problem = pd.read_csv('icu_problem_text_icu_only_20180801.csv')
patient_blood = pd.read_csv('icu_blood_test_processed_20180801.csv')
patient_vital = pd.read_csv('icu_vital_processed_20180801.csv')

In [3]:
#patinet_problemから必要な列の抽出
patient_disease = patient_problem.loc[:, [ 'pID','ent_date', 'ent_time', 'ent_disease']]
patient_disease.head()

,pID,ent_date,ent_time,ent_disease
0,5687930,2017/06/09,21:19:58,誤嚥性肺炎
1,4756415,2017/04/02,13:38:08,心不全 胸水
2,5663691,2017/04/02,18:19:07,胸椎破裂（11・12）
3,4379805,2017/04/02,21:52:27,心不全
4,6524858,2017/04/03,00:20:41,第5.6頸椎骨折


##### 必要な項目を血液データとバイタルデータから抽出する

In [4]:
#patient_bloodから列の抽出
patient_blood_pickup = patient_blood.loc[:, ['date', 'Time', 'pID', 'pO2', 'WBC', 'Hb', 'PLT', 'AST', 'ALT', 'Cre', 'Na', 'K', 'CRP', 'PTINR', 'Ddimer','BNP']]
patient_blood_pickup

,date,Time,pID,pO2,WBC,Hb,PLT,AST,ALT,Cre,Na,K,CRP,PTINR,Ddimer,BNP
0,2017/02/07,NaN,5687930,NaN,86.0,12.6,13.2,539.0,212.0,1.50,135.0,6.5,0.94,1.35,130.4,30.6
1,2017/04/02,NaN,4756415,NaN,144.0,15.3,18.5,59.0,71.0,1.10,143.0,5.1,0.20,1.41,4.0,557.0
2,2017/04/03,NaN,4756415,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,NaN,0.0,0.0
3,2017/04/02,NaN,5663691,NaN,64.0,13.5,18.3,40.0,24.0,1.00,144.0,3.9,0.06,0.97,60.2,73.7
4,2017/04/03,NaN,5663691,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.04,45.1,75.7
5,2017/04/04,NaN,5663691,NaN,112.0,8.4,8.6,77.0,32.0,0.80,142.0,4.5,7.90,0.87,20.4,NaN
6,2017/04/05,NaN,5663691,NaN,83.0,7.9,8.6,66.0,27.0,0.70,142.0,4.3,5.18,0.85,16.7,NaN
7,2017/04/06,NaN,5663691,NaN,65.0,8.5,11.4,52.0,25.0,0.60,138.0,4.3,2.84,0.91,29.1,NaN
8,2017/04/07,NaN,5663691,NaN,66.0,6.5,6.6,23.0,14.0,0.70,138.0,4.8,1.06,NaN,NaN,NaN
9,2017/04/08,NaN,5663691,NaN,83.0,11.1,11.8,30.0,18.0,0.60,138.0,4.6,3.44,NaN,NaN,NaN


##### 必要な項目を抜き出したのち、各項目についてpIDとdateと一緒に抽出する
##### 血液データについては、PTINRとDdimerとBNPが欠損値を含み、それ以外の項目は含んでいなかったので、その他の項目は一括で抽出した

In [17]:
patient_vital = patient_vital.loc[:, ['pID', 'date', 'time','BT', 'PR', 'RR']]
pID_date_time_BT = patient_vital[['pID', 'date', 'time', 'BT']]
pID_date_time_PR = patient_vital[['pID', 'date', 'time', 'PR']]
pID_date_time_RR = patient_vital[['pID', 'date', 'time', 'RR']]
pID_date_time_BT

,pID,date,time,BT
0,738587,2018/02/23,00:00:00,38.5
1,738587,2018/03/01,06:35:00,36.2
2,738587,2018/03/01,14:56:00,36.3
3,738587,2018/03/02,06:23:00,35.7
4,738587,2018/03/02,13:33:00,36.9
5,738587,2018/03/03,06:14:00,36.6
6,4711634,2018/03/21,06:24:00,36.8
7,4711634,2018/03/21,11:19:00,36.9
8,4711634,2018/03/21,21:38:00,NaN
9,4711634,2018/03/25,06:21:00,37.3


In [18]:
date_pID_PTINR = patient_blood_pickup[['date', 'pID', 'PTINR']]
date_pID_Ddimer = patient_blood_pickup[['date', 'pID', 'Ddimer']]
date_pID_BNP = patient_blood_pickup[['date', 'pID', 'BNP']]
date_pID_BloodOthers = patient_blood_pickup[['date', 'pID', 'WBC', 'Hb', 'PLT', 'AST', 'ALT', 'Cre', 'Na', 'K', 'CRP']]
date_pID_PTINR

,date,pID,PTINR
0,2017/02/07,5687930,1.35
1,2017/04/02,4756415,1.41
2,2017/04/03,4756415,NaN
3,2017/04/02,5663691,0.97
4,2017/04/03,5663691,1.04
5,2017/04/04,5663691,0.87
6,2017/04/05,5663691,0.85
7,2017/04/06,5663691,0.91
8,2017/04/07,5663691,NaN
9,2017/04/08,5663691,NaN


##### バイタルデータに関して、入室時刻と検査時刻の差分を計算できるように、それぞれdatetime型に変換する

In [19]:
#date, time, ent_date, ent_timeをdatetime型に変換
BT_Datetime = pd.to_datetime(pID_date_time_BT['date'] + pID_date_time_BT['time'], format = '%Y/%m/%d%H:%M:%S')
PR_Datetime = pd.to_datetime(pID_date_time_PR['date'] + pID_date_time_PR['time'], format = '%Y/%m/%d%H:%M:%S')
RR_Datetime = pd.to_datetime(pID_date_time_RR['date'] + pID_date_time_RR['time'], format = '%Y/%m/%d%H:%M:%S')
ent_datetime = pd.to_datetime(patient_disease['ent_date'] + patient_disease['ent_time'], format = '%Y/%m/%d%H:%M:%S')
BT_Datetime

0       2018-02-23 00:00:00
1       2018-03-01 06:35:00
2       2018-03-01 14:56:00
3       2018-03-02 06:23:00
4       2018-03-02 13:33:00
5       2018-03-03 06:14:00
6       2018-03-21 06:24:00
7       2018-03-21 11:19:00
8       2018-03-21 21:38:00
9       2018-03-25 06:21:00
10      2018-03-25 13:54:00
11      2018-02-23 00:00:00
12      2018-03-01 06:04:00
13      2018-03-01 13:34:00
14      2018-03-01 13:36:00
15      2018-03-02 06:40:00
16      2018-03-02 13:45:00
17      2018-03-03 06:13:00
18      2018-03-03 14:16:00
19      2018-03-04 04:30:00
20      2018-03-04 08:00:00
21      2018-03-04 14:00:00
22      2018-03-05 06:53:00
23      2018-03-05 13:14:00
24      2018-03-06 07:05:00
25      2018-03-21 06:11:00
26      2018-03-21 08:00:00
27      2018-03-21 20:24:00
28      2018-03-21 03:00:00
29      2018-03-21 06:32:00
                ...        
16564   2018-07-30 22:15:00
16565   2018-07-30 23:37:00
16566   2018-07-31 00:09:00
16567   2018-07-31 02:11:00
16568   2018-07-31 0

##### 上でdatetime型に変換した各データを、各項目ごとの抽出した元のファイルに結合する

In [20]:
pID_datetime_BT = pd.concat([pID_date_time_BT, BT_Datetime], axis = 1)
pID_datetime_PR = pd.concat([pID_date_time_PR, PR_Datetime], axis = 1)
pID_datetime_RR = pd.concat([pID_date_time_RR, RR_Datetime], axis = 1)
patient_disease_datetime = pd.concat([patient_disease, ent_datetime], axis = 1)
pID_datetime_BT.columns = ['pID', 'date', 'time', 'BT', 'datetime']
pID_datetime_PR.columns = ['pID', 'date', 'time', 'PR', 'datetime']
pID_datetime_RR.columns = ['pID', 'date', 'time', 'RR', 'datetime']
patient_disease_datetime.columns = ['pID', 'ent_date', 'ent_time', 'ent_disease', 'ent_datetime']
pID_datetime_BT

,pID,date,time,BT,datetime
0,738587,2018/02/23,00:00:00,38.5,2018-02-23 00:00:00
1,738587,2018/03/01,06:35:00,36.2,2018-03-01 06:35:00
2,738587,2018/03/01,14:56:00,36.3,2018-03-01 14:56:00
3,738587,2018/03/02,06:23:00,35.7,2018-03-02 06:23:00
4,738587,2018/03/02,13:33:00,36.9,2018-03-02 13:33:00
5,738587,2018/03/03,06:14:00,36.6,2018-03-03 06:14:00
6,4711634,2018/03/21,06:24:00,36.8,2018-03-21 06:24:00
7,4711634,2018/03/21,11:19:00,36.9,2018-03-21 11:19:00
8,4711634,2018/03/21,21:38:00,NaN,2018-03-21 21:38:00
9,4711634,2018/03/25,06:21:00,37.3,2018-03-25 06:21:00


##### datetime型に変換した時刻データを結合した上のファイルと、病名ファイルをkeyをpIDとしてmergeする

In [8]:
pID_disease_datetime_BT = pd.merge(pID_datetime_BT, patient_disease_datetime, how = 'inner', on = 'pID')
pID_disease_datetime_PR = pd.merge(pID_datetime_PR, patient_disease_datetime, how = 'inner', on = 'pID')
pID_disease_datetime_RR = pd.merge(pID_datetime_RR, patient_disease_datetime, how = 'inner', on = 'pID')
pID_disease_datetime_RR

,pID,date,time,RR,datetime,ent_date,ent_time,ent_disease,ent_datetime
0,738587,2018/02/23,00:00:00,NaN,2018-02-23 00:00:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14
1,738587,2018/03/01,06:35:00,NaN,2018-03-01 06:35:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14
2,738587,2018/03/01,14:56:00,NaN,2018-03-01 14:56:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14
3,738587,2018/03/02,06:23:00,NaN,2018-03-02 06:23:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14
4,738587,2018/03/02,13:33:00,NaN,2018-03-02 13:33:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14
5,738587,2018/03/03,06:14:00,NaN,2018-03-03 06:14:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14
6,4711634,2018/03/21,06:24:00,NaN,2018-03-21 06:24:00,2018/01/23,13:07:47,食道悪性黒色腫 肺炎,2018-01-23 13:07:47
7,4711634,2018/03/21,11:19:00,NaN,2018-03-21 11:19:00,2018/01/23,13:07:47,食道悪性黒色腫 肺炎,2018-01-23 13:07:47
8,4711634,2018/03/21,21:38:00,NaN,2018-03-21 21:38:00,2018/01/23,13:07:47,食道悪性黒色腫 肺炎,2018-01-23 13:07:47
9,4711634,2018/03/25,06:21:00,NaN,2018-03-25 06:21:00,2018/01/23,13:07:47,食道悪性黒色腫 肺炎,2018-01-23 13:07:47


##### バイタルデータの各項目に対して、入室時刻との差分を計算する
##### 最も入室時刻に近い値を抽出したいので、差分の絶対値をとる
##### 絶対値をとったデータを上のファイルに結合する

In [ ]:
time_interval_BT = (pID_disease_datetime_BT['datetime'] - pID_disease_datetime_BT['ent_datetime']). astype('timedelta64[m]').abs()
time_interval_PR = (pID_disease_datetime_PR['datetime'] - pID_disease_datetime_PR['ent_datetime']). astype('timedelta64[m]').abs()
time_interval_RR = (pID_disease_datetime_RR['datetime'] - pID_disease_datetime_RR['ent_datetime']). astype('timedelta64[m]').abs()
pID_disease_BT_interval = pd.concat([pID_disease_datetime_BT, time_interval_BT], axis = 1)
pID_disease_PR_interval = pd.concat([pID_disease_datetime_PR, time_interval_PR], axis = 1)
pID_disease_RR_interval = pd.concat([pID_disease_datetime_RR, time_interval_RR], axis = 1)

##### 列名の変更

In [ ]:
pID_disease_BT_interval.columns = ['pID', 'date', 'time', 'BT', 'datetime', 'ent_date', 'ent_time', 'ent_disease', 'ent_datetime', 'time_interval_BT']
pID_disease_PR_interval.columns = ['pID', 'date', 'time', 'PR', 'datetime', 'ent_date', 'ent_time', 'ent_disease', 'ent_datetime', 'time_interval_PR']
pID_disease_RR_interval.columns = ['pID', 'date', 'time', 'RR', 'datetime', 'ent_date', 'ent_time', 'ent_disease', 'ent_datetime', 'time_interval_RR']

##### 入室時刻に最も近い検査値が欠損値で無いように、最小値を取得する前に欠損値をdropする

In [ ]:
pID_disease_BT_interval = pID_disease_BT_interval.dropna(subset = ['BT'])
pID_disease_PR_interval = pID_disease_PR_interval.dropna(subset = ['PR'])
pID_disease_RR_interval = pID_disease_RR_interval.dropna(subset = ['RR'])

In [9]:
pID_disease_BT_interval

,pID,date,time,BT,datetime,ent_date,ent_time,ent_disease,ent_datetime,time_interval_BT
0,738587,2018/02/23,00:00:00,38.5,2018-02-23 00:00:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14,89412.0
1,738587,2018/03/01,06:35:00,36.2,2018-03-01 06:35:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14,98447.0
2,738587,2018/03/01,14:56:00,36.3,2018-03-01 14:56:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14,98948.0
3,738587,2018/03/02,06:23:00,35.7,2018-03-02 06:23:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14,99875.0
4,738587,2018/03/02,13:33:00,36.9,2018-03-02 13:33:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14,100305.0
5,738587,2018/03/03,06:14:00,36.6,2018-03-03 06:14:00,2017/12/22,21:47:14,CPA蘇生後 \n肺血栓塞栓症,2017-12-22 21:47:14,101306.0
6,4711634,2018/03/21,06:24:00,36.8,2018-03-21 06:24:00,2018/01/23,13:07:47,食道悪性黒色腫 肺炎,2018-01-23 13:07:47,81676.0
7,4711634,2018/03/21,11:19:00,36.9,2018-03-21 11:19:00,2018/01/23,13:07:47,食道悪性黒色腫 肺炎,2018-01-23 13:07:47,81971.0
9,4711634,2018/03/25,06:21:00,37.3,2018-03-25 06:21:00,2018/01/23,13:07:47,食道悪性黒色腫 肺炎,2018-01-23 13:07:47,87433.0
10,4711634,2018/03/25,13:54:00,36.7,2018-03-25 13:54:00,2018/01/23,13:07:47,食道悪性黒色腫 肺炎,2018-01-23 13:07:47,87886.0


##### 上で求めた時間差に対して最小値を取得する

In [10]:
time_interval_BT_min = pID_disease_BT_interval.groupby(['pID'])['time_interval_BT'].min()
time_interval_PR_min = pID_disease_PR_interval.groupby(['pID'])['time_interval_PR'].min()
time_interval_RR_min = pID_disease_RR_interval.groupby(['pID'])['time_interval_RR'].min()
time_interval_PR_min_dataframe = pd.DataFrame(time_interval_PR_min)
time_interval_PR_min_dataframe.query('pID == 6142751')

,time_interval_PR
pID,
6142751,7.0


##### 上で取得した時間差の最小値データを元の血液データファイルに結合する

In [11]:
pID_disease_BT_intervalmin = pd.merge(time_interval_BT_min, pID_disease_BT_interval, how = 'left', on = ['pID', 'time_interval_BT'])
pID_disease_PR_intervalmin = pd.merge(time_interval_PR_min, pID_disease_PR_interval, how = 'inner', on = ['pID', 'time_interval_PR'])
pID_disease_RR_intervalmin = pd.merge(time_interval_RR_min, pID_disease_RR_interval, how = 'left', on =['pID',  'time_interval_RR'])
pID_disease_RR_intervalmin

,pID,time_interval_RR,date,time,RR,datetime,ent_date,ent_time,ent_disease,ent_datetime
0,127820,7.0,2018/07/07,17:55:00,8.0,2018-07-07 17:55:00,2018/07/07,17:47:58,脱水、CO2ナルコーシス,2018-07-07 17:47:58
1,306430,161.0,2018/07/08,16:29:00,26.0,2018-07-08 16:29:00,2018/07/08,19:09:48,S状結腸穿孔,2018-07-08 19:09:48
2,400695,12.0,2018/04/17,20:41:00,15.0,2018-04-17 20:41:00,2018/04/17,20:28:55,食道癌,2018-04-17 20:28:55
3,435983,131.0,2018/07/28,17:10:00,26.0,2018-07-28 17:10:00,2018/07/28,19:20:57,S状結腸捻転術後,2018-07-28 19:20:57
4,453006,15.0,2018/03/03,22:10:00,20.0,2018-03-03 22:10:00,2018/03/03,21:54:17,肺炎・心不全,2018-03-03 21:54:17
5,485887,5.0,2018/07/25,09:52:00,19.0,2018-07-25 09:52:00,2018/07/25,09:47:00,CPA蘇生後,2018-07-25 09:47:00
6,553113,28.0,2018/07/04,16:00:00,24.0,2018-07-04 16:00:00,2018/07/04,16:27:49,顕微鏡的多発血管炎,2018-07-04 16:27:49
7,573304,30.0,2018/06/01,17:19:00,18.0,2018-06-01 17:19:00,2018/06/01,16:48:19,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19
8,602684,6489.0,2018/03/04,04:40:00,20.0,2018-03-04 04:40:00,2018/02/27,16:30:53,上腸間膜動脈塞栓,2018-02-27 16:30:53
9,632818,10.0,2018/03/19,19:45:00,13.0,2018-03-19 19:45:00,2018/03/19,19:54:16,出血性胃潰瘍,2018-03-19 19:54:16


##### 必要な列の抽出

In [12]:
pID_disease_BT_pickup = pID_disease_BT_intervalmin[['pID', 'BT', 'ent_disease', 'ent_datetime']]
pID_disease_PR_pickup = pID_disease_PR_intervalmin[['pID', 'PR', 'ent_disease', 'ent_datetime']]
pID_disease_RR_pickup = pID_disease_RR_intervalmin[['pID', 'RR', 'ent_disease', 'ent_datetime']]

##### 各項目抽出した３つのファイルを結合する

In [13]:
pID_disease_vital = pd.merge(pID_disease_BT_pickup, pID_disease_PR_pickup, how = 'outer', on = 'pID')
pID_disease_vital_second = pd.merge(pID_disease_vital, pID_disease_RR_pickup, how = 'outer', on = 'pID')

##### 病名、各データに関して重複があったので、重複の削除

In [15]:
pID_disease_vital_second_duplicate = pID_disease_vital_second.drop_duplicates()

In [16]:
pID_disease_vital_second_duplicate = pID_disease_vital_second_duplicate.dropna(subset = ['ent_disease'])
pID_disease_vital_second_duplicate

,pID,BT,ent_disease_x,ent_datetime_x,PR,ent_disease_y,ent_datetime_y,RR,ent_disease,ent_datetime
0,127820,36.1,脱水、CO2ナルコーシス,2018-07-07 17:47:58,66.0,脱水、CO2ナルコーシス,2018-07-07 17:47:58,8.0,脱水、CO2ナルコーシス,2018-07-07 17:47:58
1,306430,37.1,S状結腸穿孔,2018-07-08 19:09:48,60.0,S状結腸穿孔,2018-07-08 19:09:48,26.0,S状結腸穿孔,2018-07-08 19:09:48
2,400695,38.1,食道癌,2018-04-17 20:28:55,68.0,食道癌,2018-04-17 20:28:55,15.0,食道癌,2018-04-17 20:28:55
3,435983,38.0,S状結腸捻転術後,2018-07-28 19:20:57,134.0,S状結腸捻転術後,2018-07-28 19:20:57,26.0,S状結腸捻転術後,2018-07-28 19:20:57
4,453006,36.3,肺炎・心不全,2018-03-03 21:54:17,77.0,肺炎・心不全,2018-03-03 21:54:17,20.0,肺炎・心不全,2018-03-03 21:54:17
5,485887,38.3,CPA蘇生後,2018-07-25 09:47:00,110.0,CPA蘇生後,2018-07-25 09:47:00,19.0,CPA蘇生後,2018-07-25 09:47:00
6,553113,34.6,顕微鏡的多発血管炎,2018-07-04 16:27:49,110.0,顕微鏡的多発血管炎,2018-07-04 16:27:49,24.0,顕微鏡的多発血管炎,2018-07-04 16:27:49
7,573304,36.4,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19,144.0,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19,18.0,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19
8,602684,37.6,上腸間膜動脈塞栓,2018-02-27 16:30:53,91.0,上腸間膜動脈塞栓,2018-02-27 16:30:53,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53
9,632818,37.4,出血性胃潰瘍,2018-03-19 19:54:16,109.0,出血性胃潰瘍,2018-03-19 19:54:16,13.0,出血性胃潰瘍,2018-03-19 19:54:16


In [18]:
pID_disease_vital_second_duplicate_drop = pID_disease_vital_second_duplicate.drop(['ent_disease_x', 'ent_datetime_x', 'ent_disease_y', 'ent_datetime_y'], axis = 1)
pID_disease_vital_second_duplicate_drop

,pID,BT,PR,RR,ent_disease,ent_datetime
0,127820,36.1,66.0,8.0,脱水、CO2ナルコーシス,2018-07-07 17:47:58
1,306430,37.1,60.0,26.0,S状結腸穿孔,2018-07-08 19:09:48
2,400695,38.1,68.0,15.0,食道癌,2018-04-17 20:28:55
3,435983,38.0,134.0,26.0,S状結腸捻転術後,2018-07-28 19:20:57
4,453006,36.3,77.0,20.0,肺炎・心不全,2018-03-03 21:54:17
5,485887,38.3,110.0,19.0,CPA蘇生後,2018-07-25 09:47:00
6,553113,34.6,110.0,24.0,顕微鏡的多発血管炎,2018-07-04 16:27:49
7,573304,36.4,144.0,18.0,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19
8,602684,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53
9,632818,37.4,109.0,13.0,出血性胃潰瘍,2018-03-19 19:54:16


##### 血液データに関してもバイタルデータの時と同様、datetime型に変更して入室時刻との差分を計算後、欠損値をdropしたのち、最小値を取得する

##### dateをdatetime型に変更する

In [19]:
PTINR_date = pd.to_datetime(date_pID_PTINR['date'], format = '%Y/%m/%d')
Ddimer_date = pd.to_datetime(date_pID_Ddimer['date'], format = '%Y/%m/%d')
BNP_date = pd.to_datetime(date_pID_BNP['date'], format = '%Y/%m/%d')
BloodOthers_date = pd.to_datetime(date_pID_BloodOthers['date'], format = '%Y/%m/%d')

##### datetime型に変更する前のdateをdropする

In [20]:
date_pID_PTINR_drop = date_pID_PTINR.drop(['date'], axis = 1)
date_pID_Ddimer_drop = date_pID_Ddimer.drop(['date'], axis = 1)
date_pID_BNP_drop = date_pID_BNP.drop(['date'], axis = 1)
date_pID_BloodOthers_drop = date_pID_BloodOthers.drop(['date'], axis = 1)

In [21]:
date_pID_PTINR_merge = pd.concat([date_pID_PTINR_drop, PTINR_date], axis = 1)
date_pID_Ddimer_merge = pd.concat([date_pID_Ddimer_drop, Ddimer_date], axis = 1)
date_pID_BNP_merge = pd.concat([date_pID_BNP_drop, BNP_date], axis = 1)
date_pID_BloodOthers_merge = pd.concat([date_pID_BloodOthers_drop, BloodOthers_date], axis = 1)
date_pID_PTINR_merge

,pID,PTINR,date
0,5687930,1.35,2017-02-07
1,4756415,1.41,2017-04-02
2,4756415,NaN,2017-04-03
3,5663691,0.97,2017-04-02
4,5663691,1.04,2017-04-03
5,5663691,0.87,2017-04-04
6,5663691,0.85,2017-04-05
7,5663691,0.91,2017-04-06
8,5663691,NaN,2017-04-07
9,5663691,NaN,2017-04-08


##### datatime型に変更した上のデータを前に作成したバイタルデータに結合する

In [22]:
pID_vital_PTINR = pd.merge(date_pID_PTINR_merge, pID_disease_vital_second_duplicate_drop, how = 'inner', on = 'pID')
pID_vital_Ddimer = pd.merge(date_pID_Ddimer_merge, pID_disease_vital_second_duplicate_drop, how = 'inner', on = 'pID')
pID_vital_BNP = pd.merge(date_pID_BNP_merge, pID_disease_vital_second_duplicate_drop, how = 'inner', on = 'pID')
pID_vital_BloodOthers = pd.merge(date_pID_BloodOthers_merge, pID_disease_vital_second_duplicate_drop, how = 'inner', on = 'pID')
pID_vital_PTINR

,pID,PTINR,date,BT,PR,RR,ent_disease,ent_datetime
0,1263682,0.97,2017-06-27,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
1,1263682,0.97,2017-06-28,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
2,1263682,0.97,2017-06-27,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
3,1263682,0.97,2017-06-28,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
4,1263682,0.97,2017-06-28,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
5,1263682,1.02,2017-06-29,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
6,1263682,0.93,2017-06-30,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
7,1263682,1.19,2018-04-29,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
8,1263682,1.20,2018-04-30,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19
9,1263682,1.06,2018-05-01,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19


##### 入室時刻と検査時の時刻の差分を計算して、欠損値をdropする

In [26]:
#ent_datetimeとdateから入室してから計測するまでの時間を計算し、最も入室時刻に近い値を抽出する
pID_vital_PTINR['blood_time_interval'] = (pID_vital_PTINR['date'] - pID_vital_PTINR['ent_datetime']).astype('timedelta64[D]').abs()
pID_vital_Ddimer['blood_time_interval'] = (pID_vital_Ddimer['date'] - pID_vital_Ddimer['ent_datetime']).astype('timedelta64[D]').abs()
pID_vital_BNP['blood_time_interval'] = (pID_vital_BNP['date'] - pID_vital_BNP['ent_datetime']).astype('timedelta64[D]').abs()
pID_vital_BloodOthers['blood_time_interval'] = (pID_vital_BloodOthers['date'] - pID_vital_BloodOthers['ent_datetime']).astype('timedelta64[D]').abs()

#欠損値をdropする
pID_vital_PTINR_drop = pID_vital_PTINR.dropna(subset = ['PTINR'])
pID_vital_Ddimer_drop = pID_vital_Ddimer.dropna(subset = ['Ddimer'])
pID_vital_BNP_drop = pID_vital_BNP.dropna(subset = ['BNP'])
pID_vital_BloodOthers_drop = pID_vital_BloodOthers.dropna(subset = ['WBC', 'Hb', 'PLT', 'AST', 'ALT', 'Cre', 'Na', 'K', 'CRP'])
pID_vital_BloodOthers_drop

,pID,WBC,Hb,PLT,AST,ALT,Cre,Na,K,CRP,date,BT,PR,RR,ent_disease,ent_datetime,blood_time_interval
0,1263682,83.0,8.9,19.0,15.0,9.0,3.90,139.0,4.1,2.78,2017-06-27,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,307.0
1,1263682,87.0,9.3,20.8,14.0,10.0,4.50,139.0,3.9,3.00,2017-06-28,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,306.0
2,1263682,83.0,8.9,19.0,15.0,9.0,3.90,139.0,4.1,2.78,2017-06-27,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,307.0
3,1263682,87.0,9.3,20.8,14.0,10.0,4.50,139.0,3.9,3.00,2017-06-28,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,306.0
4,1263682,87.0,9.3,20.8,14.0,10.0,4.50,139.0,3.9,3.00,2017-06-28,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,306.0
5,1263682,59.0,9.3,19.1,17.0,9.0,3.90,139.0,4.2,3.24,2017-06-29,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,305.0
6,1263682,69.0,9.6,22.3,18.0,13.0,5.30,138.0,4.3,2.15,2017-06-30,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,304.0
7,1263682,348.0,7.1,32.2,15.0,17.0,5.42,141.0,4.2,39.33,2018-04-29,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,1.0
8,1263682,287.0,6.3,28.0,11.0,11.0,5.88,140.0,4.4,42.80,2018-04-30,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,0.0
9,1263682,214.0,8.9,23.7,34.0,15.0,3.95,139.0,3.8,29.20,2018-05-01,38.2,72.0,24.0,腎盂腎炎 Septic emboli,2018-04-29 19:42:19,1.0


##### 上の差分の最小値を取得する

In [27]:
#blood_time_interval_min = patient_vital_blood_merge.groupby(['pID'])['blood_time_interval'].min()
#patient_vital_blood_merge = pd.merge(blood_time_interval_min, patient_vital_blood_merge, how = 'inner', on = 'blood_time_interval')
#patient_vital_blood_merge
pID_vital_PTINR_min = pID_vital_PTINR.groupby(['pID'])['blood_time_interval'].min()
pID_vital_Ddimer_min = pID_vital_Ddimer.groupby(['pID'])['blood_time_interval'].min()
pID_vital_BNP_min = pID_vital_BNP.groupby(['pID'])['blood_time_interval'].min()
pID_vital_BloodOthers_min = pID_vital_BloodOthers.groupby(['pID'])['blood_time_interval'].min()

In [31]:
pID_vital_PTINR_min_merge = pd.merge(pID_vital_PTINR_min, pID_vital_PTINR_drop, how = 'inner', on = ['pID', 'blood_time_interval'])
pID_vital_Ddimer_min_merge = pd.merge(pID_vital_Ddimer_min, pID_vital_Ddimer_drop, how = 'inner', on = ['pID', 'blood_time_interval'])
pID_vital_BNP_min_merge = pd.merge(pID_vital_BNP_min, pID_vital_BNP_drop, how = 'inner', on = ['pID', 'blood_time_interval'])
pID_vital_BloodOthers_min_merge = pd.merge(pID_vital_BloodOthers_min, pID_vital_BloodOthers_drop, how = 'inner', on = ['pID', 'blood_time_interval'])


##### 重複を削除する

In [36]:
pID_vital_PTINR_min_merge_duplicate = pID_vital_PTINR_min_merge.drop_duplicates()
pID_vital_Ddimer_min_merge_duplicate = pID_vital_Ddimer_min_merge.drop_duplicates()
pID_vital_BNP_min_merge_duplicate = pID_vital_BNP_min_merge.drop_duplicates()
pID_vital_BloodOthers_min_merge_duplicate = pID_vital_BloodOthers_min_merge.drop_duplicates()
pID_vital_BloodOthers_min_merge_duplicate

,pID,blood_time_interval,WBC,Hb,PLT,AST,ALT,Cre,Na,K,CRP,date,BT,PR,RR,ent_disease,ent_datetime
0,127820,0.0,93.0,12.9,14.1,52.0,54.0,2.08,148.0,4.7,0.22,2018-07-08,36.1,66.0,8.0,脱水、CO2ナルコーシス,2018-07-07 17:47:58
2,306430,0.0,95.0,11.1,23.8,58.0,49.0,0.74,140.0,4.1,10.14,2018-07-09,37.1,60.0,26.0,S状結腸穿孔,2018-07-08 19:09:48
4,400695,0.0,0.0,0.0,0.0,75.0,43.0,0.77,140.0,4.3,10.68,2018-04-18,38.1,68.0,15.0,食道癌,2018-04-17 20:28:55
5,435983,0.0,105.0,13.2,20.2,26.0,19.0,0.78,142.0,3.6,14.18,2018-07-29,38.0,134.0,26.0,S状結腸捻転術後,2018-07-28 19:20:57
7,453006,0.0,113.0,9.0,26.9,42.0,15.0,1.10,131.0,4.4,11.40,2018-03-04,36.3,77.0,20.0,肺炎・心不全,2018-03-03 21:54:17
9,485887,0.0,140.0,9.9,34.6,46.0,9.0,0.57,144.0,4.7,18.35,2018-07-26,38.3,110.0,19.0,CPA蘇生後,2018-07-25 09:47:00
11,553113,0.0,143.0,7.6,29.2,12.0,10.0,12.13,137.0,5.7,12.31,2018-07-05,34.6,110.0,24.0,顕微鏡的多発血管炎,2018-07-04 16:27:49
13,573304,0.0,25.0,3.7,8.8,421.0,360.0,1.33,145.0,5.2,0.16,2018-06-02,36.4,144.0,18.0,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19
15,602684,1.0,67.0,10.7,20.0,29.0,14.0,0.80,137.0,5.1,2.16,2018-02-27,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53
17,602684,1.0,128.0,10.6,23.0,0.0,0.0,0.00,133.0,0.0,0.00,2018-03-01,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53


##### 項目ごとに作成していた血液データを全て結合する

In [37]:
pID_BloodOthers_PTINR_merge = pd.merge(pID_vital_BloodOthers_min_merge_duplicate, pID_vital_PTINR_min_merge_duplicate, how = 'outer', on = 'pID')
pID_BloodOthers_PTINR_Ddimer_merge = pd.merge(pID_BloodOthers_PTINR_merge, pID_vital_Ddimer_min_merge_duplicate, how = 'outer', on = 'pID')
pID_vital_merge = pd.merge(pID_vital_BloodOthers_min_merge_duplicate, pID_vital_PTINR_min_merge_duplicate, how = 'outer', on = 'pID')
pID_vital_merge

,pID,blood_time_interval_x,WBC,Hb,PLT,AST,ALT,Cre,Na,K,...,ent_disease_x,ent_datetime_x,blood_time_interval_y,PTINR,date_y,BT_y,PR_y,RR_y,ent_disease_y,ent_datetime_y
0,127820,0.0,93.0,12.9,14.1,52.0,54.0,2.08,148.0,4.7,...,脱水、CO2ナルコーシス,2018-07-07 17:47:58,0.0,1.01,2018-07-08,36.1,66.0,8.0,脱水、CO2ナルコーシス,2018-07-07 17:47:58
1,306430,0.0,95.0,11.1,23.8,58.0,49.0,0.74,140.0,4.1,...,S状結腸穿孔,2018-07-08 19:09:48,0.0,1.15,2018-07-09,37.1,60.0,26.0,S状結腸穿孔,2018-07-08 19:09:48
2,400695,0.0,0.0,0.0,0.0,75.0,43.0,0.77,140.0,4.3,...,食道癌,2018-04-17 20:28:55,0.0,1.08,2018-04-18,38.1,68.0,15.0,食道癌,2018-04-17 20:28:55
3,435983,0.0,105.0,13.2,20.2,26.0,19.0,0.78,142.0,3.6,...,S状結腸捻転術後,2018-07-28 19:20:57,0.0,1.32,2018-07-29,38.0,134.0,26.0,S状結腸捻転術後,2018-07-28 19:20:57
4,453006,0.0,113.0,9.0,26.9,42.0,15.0,1.10,131.0,4.4,...,肺炎・心不全,2018-03-03 21:54:17,0.0,0.97,2018-03-04,36.3,77.0,20.0,肺炎・心不全,2018-03-03 21:54:17
5,485887,0.0,140.0,9.9,34.6,46.0,9.0,0.57,144.0,4.7,...,CPA蘇生後,2018-07-25 09:47:00,0.0,0.99,2018-07-26,38.3,110.0,19.0,CPA蘇生後,2018-07-25 09:47:00
6,553113,0.0,143.0,7.6,29.2,12.0,10.0,12.13,137.0,5.7,...,顕微鏡的多発血管炎,2018-07-04 16:27:49,0.0,1.01,2018-07-05,34.6,110.0,24.0,顕微鏡的多発血管炎,2018-07-04 16:27:49
7,573304,0.0,25.0,3.7,8.8,421.0,360.0,1.33,145.0,5.2,...,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19,0.0,1.76,2018-06-02,36.4,144.0,18.0,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19
8,602684,1.0,67.0,10.7,20.0,29.0,14.0,0.80,137.0,5.1,...,上腸間膜動脈塞栓,2018-02-27 16:30:53,1.0,0.90,2018-02-27,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53
9,602684,1.0,67.0,10.7,20.0,29.0,14.0,0.80,137.0,5.1,...,上腸間膜動脈塞栓,2018-02-27 16:30:53,1.0,0.95,2018-03-01,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53


In [38]:
pd.set_option('display.max_columns', 50)
pID_vital_merge

,pID,blood_time_interval_x,WBC,Hb,PLT,AST,ALT,Cre,Na,K,CRP,date_x,BT_x,PR_x,RR_x,ent_disease_x,ent_datetime_x,blood_time_interval_y,PTINR,date_y,BT_y,PR_y,RR_y,ent_disease_y,ent_datetime_y
0,127820,0.0,93.0,12.9,14.1,52.0,54.0,2.08,148.0,4.7,0.22,2018-07-08,36.1,66.0,8.0,脱水、CO2ナルコーシス,2018-07-07 17:47:58,0.0,1.01,2018-07-08,36.1,66.0,8.0,脱水、CO2ナルコーシス,2018-07-07 17:47:58
1,306430,0.0,95.0,11.1,23.8,58.0,49.0,0.74,140.0,4.1,10.14,2018-07-09,37.1,60.0,26.0,S状結腸穿孔,2018-07-08 19:09:48,0.0,1.15,2018-07-09,37.1,60.0,26.0,S状結腸穿孔,2018-07-08 19:09:48
2,400695,0.0,0.0,0.0,0.0,75.0,43.0,0.77,140.0,4.3,10.68,2018-04-18,38.1,68.0,15.0,食道癌,2018-04-17 20:28:55,0.0,1.08,2018-04-18,38.1,68.0,15.0,食道癌,2018-04-17 20:28:55
3,435983,0.0,105.0,13.2,20.2,26.0,19.0,0.78,142.0,3.6,14.18,2018-07-29,38.0,134.0,26.0,S状結腸捻転術後,2018-07-28 19:20:57,0.0,1.32,2018-07-29,38.0,134.0,26.0,S状結腸捻転術後,2018-07-28 19:20:57
4,453006,0.0,113.0,9.0,26.9,42.0,15.0,1.10,131.0,4.4,11.40,2018-03-04,36.3,77.0,20.0,肺炎・心不全,2018-03-03 21:54:17,0.0,0.97,2018-03-04,36.3,77.0,20.0,肺炎・心不全,2018-03-03 21:54:17
5,485887,0.0,140.0,9.9,34.6,46.0,9.0,0.57,144.0,4.7,18.35,2018-07-26,38.3,110.0,19.0,CPA蘇生後,2018-07-25 09:47:00,0.0,0.99,2018-07-26,38.3,110.0,19.0,CPA蘇生後,2018-07-25 09:47:00
6,553113,0.0,143.0,7.6,29.2,12.0,10.0,12.13,137.0,5.7,12.31,2018-07-05,34.6,110.0,24.0,顕微鏡的多発血管炎,2018-07-04 16:27:49,0.0,1.01,2018-07-05,34.6,110.0,24.0,顕微鏡的多発血管炎,2018-07-04 16:27:49
7,573304,0.0,25.0,3.7,8.8,421.0,360.0,1.33,145.0,5.2,0.16,2018-06-02,36.4,144.0,18.0,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19,0.0,1.76,2018-06-02,36.4,144.0,18.0,仮性肝動脈瘤、消化管出血、出血性ショック,2018-06-01 16:48:19
8,602684,1.0,67.0,10.7,20.0,29.0,14.0,0.80,137.0,5.1,2.16,2018-02-27,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53,1.0,0.90,2018-02-27,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53
9,602684,1.0,67.0,10.7,20.0,29.0,14.0,0.80,137.0,5.1,2.16,2018-02-27,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53,1.0,0.95,2018-03-01,37.6,91.0,20.0,上腸間膜動脈塞栓,2018-02-27 16:30:53


In [40]:
pID_vital_merge_drop = pID_vital_merge.drop(['date_x', 'ent_datetime_x', 'blood_time_interval_y', 'blood_time_interval_x', 'date_y', 'BT_y', 'PR_y', 'RR_y', 'ent_disease_y', 'ent_datetime_y'], axis = 1)
pID_vital_merge_drop.columns = ['pID', 'WBC', 'Hb', 'PLT', 'AST', 'ALT', 'Cre', 'Na', 'K', 'CRP', 'BT', 'PR', 'RR', 'ent_disease', 'PTINR']
pID_vital_merge_drop

,pID,WBC,Hb,PLT,AST,ALT,Cre,Na,K,CRP,BT,PR,RR,ent_disease,PTINR
0,127820,93.0,12.9,14.1,52.0,54.0,2.08,148.0,4.7,0.22,36.1,66.0,8.0,脱水、CO2ナルコーシス,1.01
1,306430,95.0,11.1,23.8,58.0,49.0,0.74,140.0,4.1,10.14,37.1,60.0,26.0,S状結腸穿孔,1.15
2,400695,0.0,0.0,0.0,75.0,43.0,0.77,140.0,4.3,10.68,38.1,68.0,15.0,食道癌,1.08
3,435983,105.0,13.2,20.2,26.0,19.0,0.78,142.0,3.6,14.18,38.0,134.0,26.0,S状結腸捻転術後,1.32
4,453006,113.0,9.0,26.9,42.0,15.0,1.10,131.0,4.4,11.40,36.3,77.0,20.0,肺炎・心不全,0.97
5,485887,140.0,9.9,34.6,46.0,9.0,0.57,144.0,4.7,18.35,38.3,110.0,19.0,CPA蘇生後,0.99
6,553113,143.0,7.6,29.2,12.0,10.0,12.13,137.0,5.7,12.31,34.6,110.0,24.0,顕微鏡的多発血管炎,1.01
7,573304,25.0,3.7,8.8,421.0,360.0,1.33,145.0,5.2,0.16,36.4,144.0,18.0,仮性肝動脈瘤、消化管出血、出血性ショック,1.76
8,602684,67.0,10.7,20.0,29.0,14.0,0.80,137.0,5.1,2.16,37.6,91.0,20.0,上腸間膜動脈塞栓,0.90
9,602684,67.0,10.7,20.0,29.0,14.0,0.80,137.0,5.1,2.16,37.6,91.0,20.0,上腸間膜動脈塞栓,0.95


##### 血液データ、バイタルデータ、病名データを結合したcsvファイルの出力

In [41]:
pID_vital_merge_drop.to_csv('pID_vital_blood_entdisease.csv')